In [1]:
# Import the necessary libraries

import tensorflow as tf
import tensorflow.keras as keras

In [16]:
# This loads the EfficientNetB6 model from the Keras library
# Input Shape is the shape of the image that is input to the first layer. For example, consider an image with shape (width, height , number of channels)
# 'include_top' is set to 'False' to load the model with out the classification or dense layers. Top layers are not required as this is a segmentation problem.
# 'weights' is set to imagenet, that is, it uses the weight it learnt while training on the imagenet dataset. You can set it to None or your custom_weights.
# IMAGE_WIDTH, IMAGE_HEIGHT and CHANNELS values provided for visualization. Please change to suit your dataset.

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
CHANNELS = 3
model = tf.keras.applications.EfficientNetB6(input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, CHANNELS), 
                                             include_top=False, weights="imagenet")

165240832/165234480 [==============================] - 45s 0us/step


In [17]:
#To see the list of layers and parameters
# For EfficientNetB6, you should see
'''Total params: 40,960,143
Trainable params: 40,735,704
Non-trainable params: 224,439'''

model.summary()

Model: "efficientnetb6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_4 (Rescaling)         (None, 224, 224, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
normalization_4 (Normalization) (None, 224, 224, 3)  7           rescaling_4[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization_4[0][0]            
_____________________________________________________________________________________

In [18]:
# Importing the layers to create the decoder and complete the network
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input 
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import MeanIoU, Recall, Precision
import tensorflow_addons as tfa

In [23]:
# Defining the Convolution Block
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same", kernel_initializer="he_normal")(input)
    x = BatchNormalization()(x)
    #Used the Mish activation function as it performs better than ReLU (but is computionally expensive)
    x = tfa.activations.mish(x)
    #Comment the previous line and uncomment the next line if you limited compute resource
    #x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same", kernel_initializer="he_normal")(x)
    x = BatchNormalization()(x)
    x = tfa.activations.mish(x)
    #x = x*tf.math.tanh(tf.softplus(x)) #Mish activation in a mathematical form
    #x = Activation("relu")(x)

    return x

#Defining the Transpose Convolution Block
def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    #Use dropout only if the model is overfitting
    #x = Dropout(0.05)(x)
    x = conv_block(x, num_filters)
    return x

#Building the EfficientNetB6_UNet
def build_efficientNetB6_unet(input_shape):
    """ Input """
    inputs = Input(shape=input_shape, name='input_image')

    """ Pre-trained EfficientNetB6 Model """
    effNetB6 = tf.keras.applications.EfficientNetB6(input_tensor=inputs, include_top=False, 
                                                    weights="imagenet")
    # This Section will let you freeze and unfreeze layers. Here I have frozen all layer except
    # the last convolution block layers starting after layer 46
    for layer in effNetB6.layers[:-46]:
        layer.trainable = False
    for l in effNetB6.layers:
        print(l.name, l.trainable)

    """ Encoder """
    s1 = effNetB6.get_layer("input_image").output                   ## (512 x 512)
    s2 = effNetB6.get_layer("block1a_activation").output            ## (256 x 256)
    s3 = effNetB6.get_layer("block2a_activation").output            ## (128 x 128)
    s4 = effNetB6.get_layer("block3a_activation").output            ## (64 x 64)
    s5 = effNetB6.get_layer("block4a_activation").output            ## (32 x 32)

    """ Bridge """
    b1 = effNetB6.get_layer("block7a_activation").output  ## (16 x 16)

    """ Decoder """
    d1 = decoder_block(b1, s5, 512)                     ## (32 x 32)
    d2 = decoder_block(d1, s4, 256)                     ## (64 x 64)
    d3 = decoder_block(d2, s3, 128)                     ## (128 x 128)
    d4 = decoder_block(d3, s2, 64)                     ## (256 x 256)
    d5 = decoder_block(d4, s1, 32)                      ## (512 x 512)

    """ Output """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d5)

    model = Model(inputs, outputs, name="EfficientNetB6_U-Net")
    return model

if __name__ == "__main__":
    input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, CHANNELS)
    model = build_efficientNetB6_unet(input_shape)
    #Shows the entire EfficientNetB6_UNet Model
    model.summary()

input_image False
rescaling_9 False
normalization_9 False
stem_conv_pad False
stem_conv False
stem_bn False
stem_activation False
block1a_dwconv False
block1a_bn False
block1a_activation False
block1a_se_squeeze False
block1a_se_reshape False
block1a_se_reduce False
block1a_se_expand False
block1a_se_excite False
block1a_project_conv False
block1a_project_bn False
block1b_dwconv False
block1b_bn False
block1b_activation False
block1b_se_squeeze False
block1b_se_reshape False
block1b_se_reduce False
block1b_se_expand False
block1b_se_excite False
block1b_project_conv False
block1b_project_bn False
block1b_drop False
block1b_add False
block1c_dwconv False
block1c_bn False
block1c_activation False
block1c_se_squeeze False
block1c_se_reshape False
block1c_se_reduce False
block1c_se_expand False
block1c_se_excite False
block1c_project_conv False
block1c_project_bn False
block1c_drop False
block1c_add False
block2a_expand_conv False
block2a_expand_bn False
block2a_expand_activation False
blo

In [11]:
#Adding Model Checkpoints, Early Stopping based on Validation Loss and LR Reducer
model_path = "path/Model_Name.h5"
checkpointer = ModelCheckpoint(model_path,
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystopper = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 30,
                          verbose = 1,
                          restore_best_weights = True)

lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=0.6,
                               patience=6,
                               verbose=1,
                               min_lr=0.0001
                               #min_delta=5e-5
                              )
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=6000,
    decay_rate=0.9)
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)



In [12]:
from tensorflow.keras import backend as K
# To calculate Intersection over Union between Predicted Mask and Ground Truth
def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  
    return iou

In [13]:
smooth = 1e-5
# F1 score or Dice Coefficient
def f1_score(y_true, y_pred, smooth = 1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Soft Dice Loss
def soft_dice_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [14]:
#Compiling the model with Adam Optimizer and Metrics related to segmentation
model.compile(optimizer=optimizer,
      loss=soft_dice_loss,
      metrics=[iou_coef, Recall(), Precision(), MeanIoU(num_classes=2), f1_score])

In [15]:
# Initiate Model Training
'''history = model.fit(train_images,
                    train_masks/255,
                    validation_split=0.10,
                    epochs=EPOCHS,
                    batch_size = BATCH_SIZE,
                    callbacks = [checkpointer, earlystopper, lr_reducer])'''

'history = model.fit(train_images,\n                    train_masks/255,\n                    validation_split=0.10,\n                    epochs=EPOCHS,\n                    batch_size = BATCH_SIZE,\n                    callbacks = [checkpointer, earlystopper, lr_reducer])'